# United

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

def show_heatmap(df, title):
    # make heatmap
    heatmap = []

    for idx, row in df.iterrows():
        _, _, species, sec = row.row_id.split("_")
        if sec == "5":
            sec = "05"
        true_or_false = row.target
        heatmap.append([species,sec,true_or_false])
    
    heatmap = pd.DataFrame(heatmap, columns=["species", "sec", "True_or_False"])

    # show heamap
    fig,ax = plt.subplots(figsize=(10,5))
    cmap = sns.color_palette("Blues")
    heatmap = heatmap.pivot("species", "sec", "True_or_False")
    sns.heatmap(heatmap,ax=ax,linecolor='k',lw=1,cmap=cmap)
    plt.title(title)
    plt.show()

In [ ]:
# Load json file

import json 

submissions_json = "../input/bc2022-united/submissions.json"
with open(submissions_json) as json_file:
    submissions = json.load(json_file)

# print(json.dumps(submissions, indent=2))

submissions["submission_id"]

submission_list = list(submissions["submission_id"])
file_list = []
prob_list = []
score_list = []
model_list = []

dir_path = '../input/bc2022-united/'

for s in submission_list:
  file_list.append(dir_path + submissions['submission_id'][s]['submission_file'])
  prob_list.append(dir_path + submissions['submission_id'][s]['prob_file'])
  score_list.append(submissions['submission_id'][s]['public_score'])
  model_list.append(submissions['submission_id'][s]['model'])

print(submission_list)
print(file_list)
print(prob_list)
print(score_list)
print(model_list)

## Ensemble

In [ ]:
# Merge dataframe

for idx, file in enumerate( file_list ):
    if idx == 0:
        ensemble_df = pd.read_csv(file)
        ensemble_df = ensemble_df.rename(columns={"target": "target_"+str(idx)})
    else:
        df = pd.read_csv(file)
        df = df.rename(columns={"target": "target_"+str(idx)})
        ensemble_df = ensemble_df.merge(df, on='row_id') 


# Sum
column_list = list(ensemble_df.columns)
column_list.remove("row_id")
ensemble_df["sum"] = ensemble_df[column_list].sum(axis=1)

# New target
ensemble_df['target'] = ensemble_df['sum'] > 1

ensemble_df

In [ ]:
# Heatmap
show_heatmap(ensemble_df, 'ensemble_df')

In [ ]:
# Save submission.csv

submission_df = ensemble_df[['row_id', 'target']]
submission_df.to_csv("submission.csv", index=False)

print(len(submission_df))
submission_df.head()

## Visualization

In [ ]:
# Heatmap
for idx, data in enumerate (file_list) :
    df = pd.read_csv(file_list[idx])
    id = submission_list[idx]
    score = score_list[idx]
    show_heatmap(df, str(id) + '_' + str(score))